In [ ]:
!pip install -q python-dotenv gradio langchain langchain-community langchain-openai langchain-chroma chromadb 

In [ ]:
### Google Drive Auth Related Installation 
!pip install -q --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

## Google Docs API (Fetching Text)

In [ ]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [ ]:
TOKEN_PATH = "secrets/token.json" 
CREDENTIALS_PATH = "secrets/credentials.json"

SCOPES = [
    "https://www.googleapis.com/auth/drive.readonly",
    "https://www.googleapis.com/auth/documents.readonly",
]
FILEID = "1xWRgZ4c6BhBV97WniRY5vIWTyGlQSMljjXggKt3jfIY"

In [ ]:
creds = None

### Google Drive Authentication
def auth_google_docs():
    global creds
    if os.path.exists(TOKEN_PATH):
        creds = Credentials.from_authorized_user_file(TOKEN_PATH, SCOPES)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token: 
            creds.refresh(Request())
        else: 
            flow = InstalledAppFlow.from_client_secrets_file(
                CREDENTIALS_PATH, SCOPES
            )
            creds = flow.run_local_server(port=8080, open_browser=False)

        #Save the credentials
        with open(TOKEN_PATH, "w") as token:
            token.write(creds.to_json())

    return creds

In [ ]:
def get_docs_text():
    creds = auth_google_docs()
    ### Build Google Docs Service
    service = build("docs", "v1", credentials=creds)

    doc = service.documents().get(documentId=FILEID).execute()

    title = doc["title"]
    elements = doc["body"]["content"]

    text = ""
    for elem in elements: 
        if "paragraph" in elem: 
            for run in elem["paragraph"]["elements"]:
                if "textRun" in run: 
                    text += run["textRun"]["content"]

    return text

## Langchain LCEL (RAG) Implementation

In [ ]:
### LangChain, RAG

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain.schema import Document 
from langchain.text_splitter import CharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser 
from langchain.schema.runnable import RunnablePassthrough

In [ ]:
from dotenv import load_dotenv

In [ ]:
OPENAI_MODEL_ID ="gpt-5-mini"
db_path = "vector_db"

In [ ]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
def load_docs() -> Document: 
    docs: str = get_docs_text() 
    return [Document(page_content=docs, metadata={"source": "GoogleDocs"})]

In [ ]:
def split_text(): 
    text_splitter = CharacterTextSplitter(chunk_size=400, chunk_overlap=0)
    return text_splitter.split_documents(load_docs())

In [ ]:
def vectorize_text():
    embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)

    vector_store = Chroma.from_documents(
        documents=split_text(),
        embedding=embeddings,
        persist_directory=db_path
    )

    return vector_store

In [ ]:
def get_prompt() -> ChatPromptTemplate: 
    
    system_template = """You are a top-tier SEO strategist helping with small business SEO. 
    You are best recommended to use the provided context(retrieved documents) for your responses 
    unless you need extra resources outside for more comprehensive or insightful advice.
    You should be concise and professional.
    
    Make sure to format headings and sub headings in bold in your markdown responses:
    example: heading and sub headings in **bold**.
    
    Below is the context: 
    <context>
    {context}
    <context>
    """
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_template),
        ('human', "{question}")
    ])

    return prompt

In [ ]:
def chain_rag_elements(vector_store, prompt): 
    llm = ChatOpenAI(model=OPENAI_MODEL_ID, streaming=True)
    retriever = vector_store.as_retriever(search_kwargs={"k": 3})
    parser = StrOutputParser() 

    rag_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt 
        | llm
        | parser
    )
    
    return rag_chain


# query = "What is SEO?" 
# chain = chain_rag_elements()

# for chunk in chain.stream(query): 
    # print(chunk, end="", flush=True)

## Controller

In [ ]:
def build_rag_workflow(): 
    ### Instantiate vector store that vectorizes text 
    ### from Google Docs(as the externa resource for RAG vector store)
    vector_db = vectorize_text()
    # print(vector_store._collection.count())

    ### Chain all necessary elements into the RAG chain
    template = get_prompt()
    rag_chain = chain_rag_elements(vector_db, template)

    return rag_chain

## UI (Gradio)

In [ ]:
import gradio as gr

In [ ]:
chain = build_rag_workflow()

def chat(query, history): 
    response = ""
    for chunk in chain.stream(query):
        response += chunk
        yield response

In [ ]:
demo = gr.ChatInterface(fn=chat, type="messages")

In [ ]:
if __name__ == "__main__":
    demo.launch()
